# Extract embdeddings and store in Pinecone 

##**Step 1: Install All the Required Pakages**

In [ ]:
!pip install langchain
!pip install pypdf
!pip install sentence_transformers
!pip install pinecone-client
!pip install huggingface_hub
!pip install pinecone-client

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.8/132.8 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.4/207.4 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 36.7/36.7 MB 36.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 4.6 MB/s eta 0:00:00
  Created wheel for llama-cpp-python: filename=llama_cpp_python-0.2.46-cp310-cp310-manylinux_2_35_x86_64.whl size=2615891 sha256=c2f9cd918cb8a249f5c9d33011fe2b53f79d954e519d5890e11e04b4ac9b1525
  Stored in directory: /root/.cache/pip/wheels/1d/63/cb/c1ce62664712ab6fd32f32b9e8015bc85e6b335477471be3e4
Successfully built llama-cpp-python


#**Step 2: Import All the Required Libraries**

In [ ]:
from langchain.document_loaders import PyPDFLoader, OnlinePDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone
from sentence_transformers import SentenceTransformer
import pinecone
import os

#**Step 3: Load the Data**

In [ ]:
pip install jq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 656.0/656.0 kB 7.2 MB/s eta 0:00:00


In [ ]:
import json
from pathlib import Path
from pprint import pprint
from langchain_community.document_loaders import JSONLoader

loader = JSONLoader(
    file_path='/content/product_data20.json',
    jq_schema='.[]',
    text_content=False)

data = loader.load()

#**Step 4: Split the Text into Chunks**

In [ ]:
from langchain.docstore.document import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)

# Process all documents in the data list
all_docs = []
for document in data:
    texts = text_splitter.split_text(document.page_content)
    docs = [Document(page_content=t) for t in texts]
    all_docs.extend(docs)

# Now you have a list of document objects for all text chunks from all documents
# in the data list:

print(len(all_docs),"the lenght of all split files")

19 the lenght of all split files


#**Step 5: Setup the Environment**

In [ ]:
os.environ["HUGGINGFACEHUB_API_TOKEN"] = "hf_ataWFxkE****************"

#**Step 6: Downlaod the Embeddings**

In [ ]:
embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')

#**Step 7: Initializing the Pinecone**

In [ ]:
import os
os.environ['PINECONE_API_KEY'] = 'key'

In [ ]:
from pinecone import Pinecone
pc = Pinecone(api_key="a17ce8c3-f********************1")
pc.list_indexes()

In [14]:
index_name='Your_index_name'

In [10]:
from langchain.vectorstores import Pinecone as PineconeStore

In [15]:
docsearch=PineconeStore.from_texts(
    [t.page_content for t in all_docs],
    embeddings,
    index_name=index_name
)

In [27]:
index = pc.Index(index_name)

In [28]:
index

In [ ]:
from sentence_transformers import SentenceTransformer
import torch

device = 'cuda' if torch.cuda.is_available() else 'cpu'

model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2', device=device)
model

## Testing the database

In [32]:
query = "show me men sweaters"

# create the query vector
xq = model.encode(query).tolist()

# # now query
xc = index.query(vector=xq, top_k=5, include_metadata=True)
xc

{'matches': [{'id': 'f095f061-d092-4b22-a140-c21534ae5809',
              'metadata': {'text': '{"PRODUCT NAME": "TOMMY HILFIGER \\u2013 '
                                   'ROUND NECK SWEATERS", "PRODUCT BRAND": '
                                   '"TOMMY HILFIGER", "PRODUCT PRICE": '
                                   '"\\u20ac245.00", "PRODUCT CATEGORY": '
                                   '"MEN\'S SWEATERS", "PRODUCT TAG": '
                                   '"SWEATERS", "PRODUCT DESCRIPTION": "round '
                                   'neck long sleeves with elastic cuffs front '
                                   'logo writing logo embroidery braided '
                                   'ribbed edges Model\\u2019s height (cm): '
                                   '185 Model size: L Compositions: '
                                   '63%WOOL37%POLYAMIDE", "PRODUCT SIZE": "L, '
                                   'S, M", "PRODUCT COLOR": "OFF-WHITE", '
                         

In [33]:
for result in xc['matches']:
    print(f"{round(result['score'], 2)}: {result['metadata']['text']}")

0.53: {"PRODUCT NAME": "TOMMY HILFIGER \u2013 ROUND NECK SWEATERS", "PRODUCT BRAND": "TOMMY HILFIGER", "PRODUCT PRICE": "\u20ac245.00", "PRODUCT CATEGORY": "MEN'S SWEATERS", "PRODUCT TAG": "SWEATERS", "PRODUCT DESCRIPTION": "round neck long sleeves with elastic cuffs front logo writing logo embroidery braided ribbed edges Model\u2019s height (cm): 185 Model size: L Compositions: 63%WOOL37%POLYAMIDE", "PRODUCT SIZE": "L, S, M", "PRODUCT COLOR": "OFF-WHITE", "PRODUCT LINK": "https://www.tuscanrhea.com/product/tommy-hilfiger-round-neck-sweaters-5/?v=38dd815e66db"}
0.49: {"PRODUCT NAME": "Jacqueline De Yong Women Knitwear", "PRODUCT BRAND": "Jacqueline De Yong", "PRODUCT PRICE": "\u20ac49.00", "PRODUCT CATEGORY": "WOMEN'S SWEATERS", "PRODUCT TAG": "SWEATERS", "PRODUCT DESCRIPTION": "Brand: Jacqueline De Yong Gender: Women Type: Knitwear Season: Fall/Winter PRODUCT DETAIL \u2022 Color: brown \u2022 Pattern: leopard \u2022 Fastening: slip on \u2022 Sleeves: long \u2022 Neckline: round neck C